In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [7]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(3, 16, 1, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, padding = "same"),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 3, 1, 1, padding = "same"),
            nn.BatchNorm2d(3),
        )
        
        self.conv01 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        identity = x
        x = self.conv0(x)
        x = identity + x
        x = nn.ReLU()(x)
        x = self.conv1(x)
        identity = x
        x = self.conv01(x)
        x = identity + x
        x = self.conv2(x)
        x = self.conv3(x)
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [8]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(16, 3, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv01): Sequential(
    (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), padding=

# Step 3 Pass in the model to start training and testing

In [9]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 2.9655
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 2.9396
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.5889
10.93935775756836
epoch: 01 | Accuracy: 10.94 Loss: 79.5688

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.9164
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.3480
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.6175
30.439952850341797
epoch: 02 | Accuracy: 30.44 Loss: 64.4133

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 1.8517
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 2.2990
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.3019
46.73008346557617
epoch: 03 | Accuracy: 46.73 Loss: 47.3269

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.4372
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.4957
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.7770
63.4958381652832
epoch: 04 | Accuracy: 63.50 Loss: 33.7476

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 0.7221
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.4807
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 1.7166
76.81331634521484
epoch: 05 | Accuracy: 76.81 Loss: 20.9625

Epoch: [6/30]	|	Step: [9/

In [10]:
print(Accy_list)

[10.93935775756836, 30.439952850341797, 46.73008346557617, 63.4958381652832, 76.81331634521484, 84.78002166748047, 92.50891876220703, 94.88703918457031, 96.78953552246094, 97.14625549316406, 98.2164077758789, 99.04875183105469, 99.40547180175781, 99.88109588623047, 97.50297546386719, 92.03329467773438, 95.7193832397461, 99.40547180175781, 99.7621841430664, 99.7621841430664, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [11]:
print(Loss_list)

[79.56881189346313, 64.4132512807846, 47.32693028450012, 33.74759203195572, 20.962539792060852, 13.060379430651665, 6.524556759744883, 4.120524813421071, 3.2047951966524124, 2.2445809226483107, 1.6944516049697995, 0.8633519385475665, 0.5665428985375911, 0.5436636576196179, 2.595123496139422, 6.700883479788899, 3.336076809093356, 0.8792650058167055, 0.3696829122491181, 0.1865905944723636, 0.05827558648888953, 0.031269402301404625, 0.017432046137400903, 0.019315436686156318, 0.01289791680756025, 0.012917325046146289, 0.00910288352315547, 0.009452043319470249, 0.006036004433553899, 0.006206956873938907]


In [12]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 76.10 %, Loss: 50.4421 
